# Similarity Splitter 

In [1]:
import os,sys, tempfile, pandas as pd, shutil, logging, glob
sys.path.append("src")

In [2]:
base_data_dir = "tmp"

In [3]:
working_dir =  "tempspliiterhajkA"
if os.path.exists(working_dir) : shutil.rmtree(working_dir)
os.makedirs(working_dir, exist_ok=True)

In [4]:
def scores_to_df(scores, details, data_set, data_type, task_type):
    df = pd.DataFrame()
    for k , v in scores.items():
        df[k] = v
        
    for k , v in details.items():
        df[k + "_detail"] = v
    
    df["data_set"] = data_set + " " + data_type
    df["task_type"] = task_type
         
    return df

In [5]:
logging.basicConfig(level="WARN", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [6]:
thresholds = [0,25,50,75,100]
ngrams = [1]
num_parts = 4

### 1. BC2GM

[Biocreative II gene mention](https://biocreative.bioinformatics.udel.edu/tasks/biocreative-ii/) overlap. Please download the test and train files for this task from the BioCreative Website.


In [7]:
bc2gm_train_file = os.path.join(base_data_dir, "train.in")
bc2gm_test_file = os.path.join(base_data_dir, "test.in")

bc2gm_train_eval_file = os.path.join(base_data_dir, "trainGENE.eval")
bc2gm_test_eval_file = os.path.join(base_data_dir, "testGENE.eval")

bc2gm_test_alteval_file = os.path.join(base_data_dir, "testALTGENE.eval")

# This is the predictions
bc2gm_predictions_file = os.path.join(base_data_dir, "bc2gm_result_test_pred.txt")

In [8]:
from bc2_gene_mention import BC2GeneMentionText 


bc2gmrun = BC2GeneMentionText()



### BC2GM : Threshold based simlarity

In [9]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_detail = bc2gmrun.run_similarity_threshold_splitter("text", bc2gm_train_file,  bc2gm_test_file, bc2gm_out_dir_text, bc2gm_test_eval_file, bc2gm_test_alteval_file,bc2gm_predictions_file ,thresholds,ngrams)
df_bc2_gm_text = pd.DataFrame(result_detail)
df_bc2_gm_text["dataset"] = "BC2GM (text)"

In [10]:
df_bc2_gm_text.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0,25.0,990,19.80,0.744872,0.688389,0.811453,BC2GM (text)
1,1,25,50.0,3706,74.12,0.823754,0.782895,0.869114,BC2GM (text)
2,1,50,75.0,257,5.14,0.870801,0.838308,0.905914,BC2GM (text)
3,1,75,NaN,47,0.94,0.784810,0.794872,0.775000,BC2GM (text)
4,-1,0,100.0,5000,100.00,0.817055,0.774819,0.864160,BC2GM (text)


In [11]:
bc2gm_out_dir_anno =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_anno, exist_ok=True)
result_detail = bc2gmrun.run_similarity_threshold_splitter("eval", bc2gm_train_eval_file,  bc2gm_test_eval_file, bc2gm_out_dir_anno, bc2gm_test_eval_file, bc2gm_test_alteval_file, bc2gm_predictions_file ,thresholds,ngrams)
df_bc2_gm_anno = pd.DataFrame(result_detail)
df_bc2_gm_anno["dataset"] = "BC2GM (anno)"

In [12]:
df_bc2_gm_anno.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0,25.0,1059,16.727215,0.844237,0.843861,0.844613,BC2GM (anno)
1,1,25,50.0,352,5.559943,0.808743,0.800000,0.817680,BC2GM (anno)
2,1,50,75.0,1565,24.719634,0.834215,0.813616,0.855885,BC2GM (anno)
3,1,75,NaN,3355,52.993208,0.854116,0.831631,0.877852,BC2GM (anno)
4,-1,0,100.0,6331,100.000000,0.817055,0.774819,0.864160,BC2GM (anno)


### BC2GM : Sorted based simlarity

In [13]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_score, result_detail = bc2gmrun.run_similarity_parts_splitter("text", bc2gm_train_file,  bc2gm_test_file, bc2gm_out_dir_text, bc2gm_test_eval_file, bc2gm_test_alteval_file,bc2gm_predictions_file ,num_parts)
df_bc2_gm_text_parts_score = pd.DataFrame(result_score)
df_bc2_gm_text_parts_score["dataset"] = "BC2GM (text)"

Exact matches Unigram, 39 / 5000
Exact matches Bigram, 26 / 5000
Exact matches Trigram, 26 / 5000


In [14]:
df_bc2_gm_text_parts_score.head()

,ngram,min,max,num,percent,f-score,precision,recall,dataset
0,1,0.000000,26.311741,1250,25.0,0.754003,0.698113,0.819620,BC2GM (text)
1,1,26.311741,31.622777,1250,25.0,0.798122,0.745178,0.859164,BC2GM (text)
2,1,31.622777,38.332594,1250,25.0,0.821266,0.782819,0.863685,BC2GM (text)
3,1,38.348249,100.000000,1250,25.0,0.858576,0.830023,0.889164,BC2GM (text)
4,-1,0.000000,100.000000,5000,100.0,0.817055,0.774819,0.864160,BC2GM (text)


## 2. SST2 Dataset

In [15]:
sst2_sentences_file = os.path.join("tmp", "datasetSentences.txt")
sst2_sentiment_labels_file  =  os.path.join("tmp", "sentiment_labels.txt")
sst2_dictionary_file  =  os.path.join("tmp", "dictionary.txt") 
sst2_datatset_split_file  =  os.path.join("tmp", "datasetSplit.txt") 


# This is the predictions
sst2_predictions_file = os.path.join(base_data_dir, "sst2-output.csv")

In [16]:
from sst2_dataset import SST2Dataset
sst2_run=SST2Dataset(sst2_sentences_file, sst2_sentiment_labels_file, sst2_datatset_split_file, sst2_dictionary_file)


2021-01-20 23:55:24,821 - sst2_dataset - WARNING - Text not found in dictionary: But in Imax 3-D , the clichÃ©s disappear into the vertiginous perspectives opened up by the photography .
2021-01-20 23:55:24,823 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- But it 's -RRB- worth recommending because of two marvelous performances by Michael Caine and Brendan Fraser .
2021-01-20 23:55:24,825 - sst2_dataset - WARNING - Text not found in dictionary: JirÃ­ Hubac 's script is a gem .
2021-01-20 23:55:24,828 - sst2_dataset - WARNING - Text not found in dictionary: You would n't call The Good Girl a date movie -LRB- an anti-date movie is more like it -RRB- , but when it 's good , it 's good and horrid .
2021-01-20 23:55:24,830 - sst2_dataset - WARNING - Text not found in dictionary: An incendiary , deeply thought-provoking look at one of the most peculiar -LRB- and peculiarly venomous -RRB- bigotries in our increasingly frightening theocracy
2021-01-20 23:55:24,832 - sst2_datas

2021-01-20 23:55:24,962 - sst2_dataset - WARNING - Text not found in dictionary: Dazzles with its fully-written characters , its determined stylishness -LRB- which always relates to characters and story -RRB- and Johnny Dankworth 's best soundtrack in years .
2021-01-20 23:55:24,963 - sst2_dataset - WARNING - Text not found in dictionary: The band 's courage in the face of official repression is inspiring , especially for aging hippies -LRB- this one included -RRB- .
2021-01-20 23:55:24,964 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Lawrence bounces -RRB- all over the stage , dancing , running , sweating , mopping his face and generally displaying the wacky talent that brought him fame in the first place .
2021-01-20 23:55:24,965 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- NÃ¦s -RRB- directed the stage version of Elling , and gets fine performances from his two leads who originated the characters on stage .
2021-01-20 23:55:24,966 - sst2_dataset -

2021-01-20 23:55:25,029 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Reno -RRB- delivers a monologue that manages to incorporate both the horror and the absurdity of the situation in a well-balanced fashion .
2021-01-20 23:55:25,031 - sst2_dataset - WARNING - Text not found in dictionary: ... the first 2\/3 of the film are incredibly captivating and insanely funny , thanks in part to interesting cinematic devices -LRB- cool visual backmasking -RRB- , a solid cast , and some wickedly sick and twisted humor ...
2021-01-20 23:55:25,031 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Haynes ' -RRB- homage to such films as `` All That Heaven Allows '' and `` Imitation of Life '' transcends them .
2021-01-20 23:55:25,032 - sst2_dataset - WARNING - Text not found in dictionary: SchÃ¼tte 's dramatic snapshot of the artist three days before his death offers an interesting bit of speculation as to the issues Brecht faced as his life drew to a close .
2021-01-20 2

2021-01-20 23:55:25,094 - sst2_dataset - WARNING - Text not found in dictionary: Lead actress GaÃ¯ , she of the impossibly long limbs and sweetly conspiratorial smile , is a towering siren .
2021-01-20 23:55:25,095 - sst2_dataset - WARNING - Text not found in dictionary: Even if you 've seen `` Stomp '' -LRB- the stage show -RRB- , you still have to see this !
2021-01-20 23:55:25,097 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- The digital effects -RRB- reminded me of Terry Gilliam 's rudimentary old Monty Python cartoons , in which he would cut out figures from drawings and photographs and paste them together .
2021-01-20 23:55:25,098 - sst2_dataset - WARNING - Text not found in dictionary: The fluid motion is astounding on any number of levels -- including the physical demands made on BÃ¼ttner -- and it implies in its wake the intractable , irreversible flow of history .
2021-01-20 23:55:25,099 - sst2_dataset - WARNING - Text not found in dictionary: This ready-made

2021-01-20 23:55:25,136 - sst2_dataset - WARNING - Text not found in dictionary: Jolting into Charleston rhythms , the story has the sizzle of old news that has finally found the right vent -LRB- accurate ?
2021-01-20 23:55:25,137 - sst2_dataset - WARNING - Text not found in dictionary: Who cares ? -RRB- .
2021-01-20 23:55:25,138 - sst2_dataset - WARNING - Text not found in dictionary: Damon brings the proper conviction to his role as -LRB- Jason Bourne -RRB- .
2021-01-20 23:55:25,139 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Barry -RRB- gives Assassin a disquieting authority .
2021-01-20 23:55:25,141 - sst2_dataset - WARNING - Text not found in dictionary: At its best -LRB- and it does have some very funny sequences -RRB- Looking for Leonard reminds you just how comically subversive silence can be .
2021-01-20 23:55:25,142 - sst2_dataset - WARNING - Text not found in dictionary: Transforms one of -LRB- Shakespeare 's -RRB- deepest tragedies into a smart new comedy

...

2021-01-20 23:55:25,528 - sst2_dataset - WARNING - Text not found in dictionary: Feels less like a change in -LRB- Herzog 's -RRB- personal policy than a half-hearted fluke .
2021-01-20 23:55:25,529 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Janey -RRB- forgets about her other obligations , leading to a tragedy which is somehow guessable from the first few minutes , maybe because it echoes the by now intolerable morbidity of so many recent movies .
2021-01-20 23:55:25,531 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Evans is -RRB- a fascinating character , and deserves a better vehicle than this facetious smirk of a movie .
2021-01-20 23:55:25,532 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- I -RRB- t 's certainly laudable that the movie deals with hot-button issues in a comedic context , but Barbershop is n't as funny as it should be .
2021-01-20 23:55:25,533 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Less a m

2021-01-20 23:55:25,570 - sst2_dataset - WARNING - Text not found in dictionary: With an unusual protagonist -LRB- a kilt-wearing Jackson -RRB- and subject matter , the improbable `` Formula 51 '' is somewhat entertaining , but it could have been much stronger .
2021-01-20 23:55:25,571 - sst2_dataset - WARNING - Text not found in dictionary: Sandra Bullock 's best dramatic performance to date -LRB- is -RRB- almost enough to lift -LRB- this -RRB- thrill-kill cat-and-mouser ... above its paint-by-numbers plot .
2021-01-20 23:55:25,572 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- The kid 's -RRB- just too bratty for sympathy , and as the film grows to its finale , his little changes ring hollow .
2021-01-20 23:55:25,572 - sst2_dataset - WARNING - Text not found in dictionary: Starts out ballsy and stylish but fails to keep it up and settles into clichÃ©s .
2021-01-20 23:55:25,573 - sst2_dataset - WARNING - Text not found in dictionary: Five screenwriters are credited wit

2021-01-20 23:55:25,613 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Creates -RRB- the worst kind of mythologizing , the kind that sacrifices real heroism and abject suffering for melodrama .
2021-01-20 23:55:25,614 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- Allen 's -RRB- been making piffle for a long while , and Hollywood Ending may be his way of saying that piffle is all that the airhead movie business deserves from him right now .
2021-01-20 23:55:25,615 - sst2_dataset - WARNING - Text not found in dictionary: After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichÃ©s , depression and bad alternative music .
2021-01-20 23:55:25,617 - sst2_dataset - WARNING - Text not found in dictionary: The film was produced by Jerry Bruckheimer and directed by Joel Schumacher , and reflects the worst of their shallow styles : wildly overproduced , inadequately motivated every step of the way and demographically targeted to 

2021-01-20 23:55:25,660 - sst2_dataset - WARNING - Text not found in dictionary: Return to Never Land is much more P.C. than the original version -LRB- no more racist portraits of Indians , for instance -RRB- , but the excitement is missing .
2021-01-20 23:55:25,661 - sst2_dataset - WARNING - Text not found in dictionary: By the end , you just do n't care whether that cold-hearted snake Petrovich -LRB- that would be Reno -RRB- gets his comeuppance .
2021-01-20 23:55:25,662 - sst2_dataset - WARNING - Text not found in dictionary: Don Michael Paul uses quick-cuts , -LRB- very -RRB- large shadows and wide-angle shots taken from a distance to hide the liberal use of a body double -LRB- for Seagal -RRB- .
2021-01-20 23:55:25,663 - sst2_dataset - WARNING - Text not found in dictionary: For something as splendid-looking as this particular film , the viewer expects something special but instead gets -LRB- sci-fi -RRB- rehash .
2021-01-20 23:55:25,664 - sst2_dataset - WARNING - Text not found i

2021-01-20 23:55:25,715 - sst2_dataset - WARNING - Text not found in dictionary: -LRB- A -RRB- poorly executed comedy .
2021-01-20 23:55:25,717 - sst2_dataset - WARNING - A 569 out of 11855 were not found in dictionary


###  SST2 : Threshold based simlarity

In [17]:

sst2_out_dir_text =  os.path.join(working_dir, "sst2_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_detail = sst2_run.run_similarity_threshold_splitter(  sst2_predictions_file ,thresholds,ngrams)
df_sst2_text = pd.DataFrame(result_detail)
df_sst2_text["dataset"] = "SST2 (text)"

In [18]:
df_sst2_text.head()

,ngram,min,max,num,percent,f-score,precision,recall,accuracy,dataset
0,1,0,25.0,269,12.658824,0.857143,0.902439,0.816176,0.862454,SST2 (text)
1,1,25,50.0,1565,73.647059,0.839974,0.871274,0.810845,0.843450,SST2 (text)
2,1,50,75.0,277,13.035294,0.851711,0.910569,0.800000,0.859206,SST2 (text)
3,1,75,NaN,14,0.658824,0.857143,0.857143,0.857143,0.857143,SST2 (text)
4,-1,0,100.0,2125,100.000000,0.843735,0.879919,0.810409,0.848000,SST2 (text)


###  SST2 : Sorted based simlarity

In [19]:
num_parts = 4

In [20]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_score, result_detail = sst2_run.run_similarity_parts_splitter(  sst2_predictions_file , num_parts)
df_sst2_text_parts_score = pd.DataFrame(result_score)
df_sst2_text_parts_score["dataset"] = "SST2 (text)"

In [21]:
df_sst2_text_parts_score.head()

,ngram,min,max,num,percent,f-score,precision,recall,accuracy,dataset
0,1,0.000000,28.347335,532,25.035294,0.834615,0.871486,0.800738,0.838346,SST2 (text)
1,1,28.347335,33.333333,532,25.035294,0.838346,0.892000,0.790780,0.838346,SST2 (text)
2,1,33.333333,42.163702,532,25.035294,0.841509,0.864341,0.819853,0.842105,SST2 (text)
3,1,42.163702,100.000000,529,24.894118,0.861856,0.893162,0.832669,0.873346,SST2 (text)
4,-1,0.000000,100.000000,2125,100.000000,0.843735,0.879919,0.810409,0.848000,SST2 (text)


## 3. BC3 Article classification

In [22]:
bc3_act_train_file = os.path.join("tmp", "bc3_act_all_records.tsv")
bc3_act_test_file = os.path.join("tmp", "bc3_act_all_records_test.tsv")


bc3_act_train_eval_file = os.path.join(base_data_dir, "bc3_act_gold_standard.tsv")
bc3_act_test_eval_file = os.path.join(base_data_dir, "bc3_act_gold_standard_test.tsv")


# This is the predictions
bc3_act_predictions_file = os.path.join(base_data_dir, "bc3act-output.csv")

In [23]:
from bc3_article_classification import BC3ArticleClassification

bc3actrun = BC3ArticleClassification()


###  BC3 Article classification : Threshold based simlarity

In [24]:

bc3act_out_dir_text =  os.path.join(working_dir, "bc3act_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_detail = bc3actrun.run_similarity_threshold_splitter( bc3_act_train_file,  bc3_act_test_file, bc3_act_test_eval_file, bc3_act_predictions_file ,thresholds,ngrams)
df_bc3_act_text = pd.DataFrame(result_detail)
df_bc3_act_text["dataset"] = "BC3ACT (text)"

/Users/aeg/envs/nlp-train-test-overlap-detector/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
df_bc3_act_text.head()

,ngram,min,max,num,percent,f-score,precision,recall,accuracy,dataset
0,1,0,25.0,2817,46.950000,0.564270,0.430233,0.819620,0.858005,BC3ACT (text)
1,1,25,50.0,3062,51.033333,0.598513,0.460000,0.856383,0.788374,BC3ACT (text)
2,1,50,75.0,120,2.000000,0.630137,0.534884,0.766667,0.775000,BC3ACT (text)
3,1,75,NaN,1,0.016667,0.000000,0.000000,0.000000,0.000000,BC3ACT (text)
4,-1,0,100.0,6000,100.000000,0.587107,0.451061,0.840659,0.820667,BC3ACT (text)


###  BC3 Article classification : Sorted based simlarity

In [26]:
num_parts = 4

In [27]:

bc2gm_out_dir_text =  os.path.join(working_dir, "bc2gm_text")
os.makedirs(bc2gm_out_dir_text, exist_ok=True)
result_score, result_detail = bc3actrun.run_similarity_parts_splitter(bc3_act_train_file,  bc3_act_test_file, bc3_act_test_eval_file, bc3_act_predictions_file ,num_parts)
df_bc3_act_text_parts_score = pd.DataFrame(result_score)
df_bc3_act_text_parts_score["dataset"] = "BC3ACT (text)"

In [28]:
df_bc3_act_text_parts_score.head()

,ngram,min,max,num,percent,f-score,precision,recall,accuracy,dataset
0,1,6.277610,20.090702,1500,25.0,0.575758,0.445312,0.814286,0.888000,BC3ACT (text)
1,1,20.095193,25.676454,1500,25.0,0.559322,0.423077,0.825000,0.826667,BC3ACT (text)
2,1,25.677981,31.913278,1500,25.0,0.602067,0.465070,0.853480,0.794667,BC3ACT (text)
3,1,31.917000,75.011317,1500,25.0,0.598109,0.460838,0.851852,0.773333,BC3ACT (text)
4,-1,0.000000,100.000000,6000,100.0,0.587107,0.451061,0.840659,0.820667,BC3ACT (text)


## Summary

In [29]:
formatters = { t:lambda x: "{:.2%}".format(x).replace("%","")  for t in {"f-score", "precision", "recall"}}
formatters['min']=lambda x: "{:.1f}".format(x)
formatters['max']=lambda x: "{:.1f}".format(x)
formatters['percent']=lambda x: "{:.1f}".format(x)


float_formatter = lambda x: "{:.2f}".format(x)

### Summary Threshold

In [30]:
df_threshold_summary = pd.concat([df_bc2_gm_text, df_bc3_act_text,df_sst2_text])

In [31]:
df_threshold_summary.sort_values(by=["dataset", "ngram", "min", "max"])

,ngram,min,max,num,percent,f-score,precision,recall,dataset,accuracy
4,-1,0,100.0,5000,100.000000,0.817055,0.774819,0.864160,BC2GM (text),NaN
0,1,0,25.0,990,19.800000,0.744872,0.688389,0.811453,BC2GM (text),NaN
1,1,25,50.0,3706,74.120000,0.823754,0.782895,0.869114,BC2GM (text),NaN
2,1,50,75.0,257,5.140000,0.870801,0.838308,0.905914,BC2GM (text),NaN
3,1,75,NaN,47,0.940000,0.784810,0.794872,0.775000,BC2GM (text),NaN
4,-1,0,100.0,6000,100.000000,0.587107,0.451061,0.840659,BC3ACT (text),0.820667
0,1,0,25.0,2817,46.950000,0.564270,0.430233,0.819620,BC3ACT (text),0.858005
1,1,25,50.0,3062,51.033333,0.598513,0.460000,0.856383,BC3ACT (text),0.788374
2,1,50,75.0,120,2.000000,0.630137,0.534884,0.766667,BC3ACT (text),0.775000
3,1,75,NaN,1,0.016667,0.000000,0.000000,0.000000,BC3ACT (text),0.000000


In [32]:
print(df_threshold_summary[["dataset", "ngram", "min", "max","num", "f-score", "precision" , "recall","accuracy" ]]
      .sort_values(by=["dataset", "ngram", "min", "max"])
      .to_latex(float_format=float_formatter, formatters=formatters, index=False))

\begin{tabular}{lrrrrrrrr}
\toprule
       dataset &  ngram &  min &   max &   num & f-score & precision & recall &  accuracy \\
\midrule
  BC2GM (text) &     -1 &  0.0 & 100.0 &  5000 &   81.71 &     77.48 &  86.42 &       nan \\
  BC2GM (text) &      1 &  0.0 &  25.0 &   990 &   74.49 &     68.84 &  81.15 &       nan \\
  BC2GM (text) &      1 & 25.0 &  50.0 &  3706 &   82.38 &     78.29 &  86.91 &       nan \\
  BC2GM (text) &      1 & 50.0 &  75.0 &   257 &   87.08 &     83.83 &  90.59 &       nan \\
  BC2GM (text) &      1 & 75.0 &   nan &    47 &   78.48 &     79.49 &  77.50 &       nan \\
 BC3ACT (text) &     -1 &  0.0 & 100.0 &  6000 &   58.71 &     45.11 &  84.07 &      0.82 \\
 BC3ACT (text) &      1 &  0.0 &  25.0 &  2817 &   56.43 &     43.02 &  81.96 &      0.86 \\
 BC3ACT (text) &      1 & 25.0 &  50.0 &  3062 &   59.85 &     46.00 &  85.64 &      0.79 \\
 BC3ACT (text) &      1 & 50.0 &  75.0 &   120 &   63.01 &     53.49 &  76.67 &      0.78 \\
 BC3ACT (text) &      1 &

### Summary Split by quartile

In [33]:
df_quartile_summary = pd.concat([df_bc2_gm_text_parts_score, df_bc3_act_text_parts_score,df_sst2_text_parts_score])

In [34]:
df_quartile_summary .sort_values(by=["dataset", "ngram","min","max" ])

,ngram,min,max,num,percent,f-score,precision,recall,dataset,accuracy
4,-1,0.000000,100.000000,5000,100.000000,0.817055,0.774819,0.864160,BC2GM (text),NaN
0,1,0.000000,26.311741,1250,25.000000,0.754003,0.698113,0.819620,BC2GM (text),NaN
1,1,26.311741,31.622777,1250,25.000000,0.798122,0.745178,0.859164,BC2GM (text),NaN
2,1,31.622777,38.332594,1250,25.000000,0.821266,0.782819,0.863685,BC2GM (text),NaN
3,1,38.348249,100.000000,1250,25.000000,0.858576,0.830023,0.889164,BC2GM (text),NaN
4,-1,0.000000,100.000000,6000,100.000000,0.587107,0.451061,0.840659,BC3ACT (text),0.820667
0,1,6.277610,20.090702,1500,25.000000,0.575758,0.445312,0.814286,BC3ACT (text),0.888000
1,1,20.095193,25.676454,1500,25.000000,0.559322,0.423077,0.825000,BC3ACT (text),0.826667
2,1,25.677981,31.913278,1500,25.000000,0.602067,0.465070,0.853480,BC3ACT (text),0.794667
3,1,31.917000,75.011317,1500,25.000000,0.598109,0.460838,0.851852,BC3ACT (text),0.773333


In [35]:
print(df_quartile_summary.query("ngram in (1,-1)")
      .sort_values(by=["dataset", "ngram","min","max" ])
      [["dataset", "min", "max", "f-score", "precision" , "recall","accuracy" ]]
      .fillna("-")
      .to_latex( float_format=float_formatter, formatters=formatters, index=False))

\begin{tabular}{lrrrrrl}
\toprule
       dataset &  min &   max & f-score & precision & recall & accuracy \\
\midrule
  BC2GM (text) &  0.0 & 100.0 &   81.71 &     77.48 &  86.42 &        - \\
  BC2GM (text) &  0.0 &  26.3 &   75.40 &     69.81 &  81.96 &        - \\
  BC2GM (text) & 26.3 &  31.6 &   79.81 &     74.52 &  85.92 &        - \\
  BC2GM (text) & 31.6 &  38.3 &   82.13 &     78.28 &  86.37 &        - \\
  BC2GM (text) & 38.3 & 100.0 &   85.86 &     83.00 &  88.92 &        - \\
 BC3ACT (text) &  0.0 & 100.0 &   58.71 &     45.11 &  84.07 &     0.82 \\
 BC3ACT (text) &  6.3 &  20.1 &   57.58 &     44.53 &  81.43 &     0.89 \\
 BC3ACT (text) & 20.1 &  25.7 &   55.93 &     42.31 &  82.50 &     0.83 \\
 BC3ACT (text) & 25.7 &  31.9 &   60.21 &     46.51 &  85.35 &     0.79 \\
 BC3ACT (text) & 31.9 &  75.0 &   59.81 &     46.08 &  85.19 &     0.77 \\
   SST2 (text) &  0.0 & 100.0 &   84.37 &     87.99 &  81.04 &     0.85 \\
   SST2 (text) &  0.0 &  28.3 &   83.46 &     87.15 &  80